# Created hyperparameter tuning job on the GUI (which now fails), attempted to run tuning job with code below.

### So far, no examples of full hyperparameter tuning code available, that I have found at least.

#### Documentation from Sagemaker avaiable [here]()

## Specify where your training and testing data are located

In [ ]:
s3_input_train = 's3://{}/{}/data/train'.format(bucket, prefix)
s3_input_test= 's3://{}/{}/data/test'.format(bucket, prefix)

## Specify your training image location (your region)

In [ ]:
training_image = containers[region]

In [ ]:
HyperParameterTuningJobConfig={
        'Strategy': 'Bayesian',
        'HyperParameterTuningJobObjective': {
            'Type': 'Minimize',
            'MetricName': 'test:RMSE' # metric could either be Bayesian or RMSE, unclear.
        },
        'ResourceLimits': {
            'MaxNumberOfTrainingJobs': 50, 
            'MaxParallelTrainingJobs': 3
        },
        'ParameterRanges': {
            'IntegerParameterRanges': [
                {
                    'Name': 'mini_batch_size',
                    'MinValue': '32',
                    'MaxValue': '50'
                },
                {
                    'Name': 'epochs',
                    'MinValue': '32',
                    'MaxValue': '50'
                },
                {
                    'Name': 'context_length',
                    'MinValue': '20',
                    'MaxValue': '50'
                },            
                {
                    'Name': 'num_cells',
                    'MinValue': '30',
                    'MaxValue': '70'
                },            
                {
                    'Name': 'num_layers',
                    'MinValue': '1',
                    'MaxValue': '5'
                },                  
            
            ],
            'ContinuousParameterRanges': [
                {
                    'Name': 'dropout_rate',
                    'MinValue': '0.00',
                    'MaxValue': '0.2'
                },
                {
                    'Name': 'learning_rate',
                    'MinValue': '0.0001',
                    'MaxValue': '0.01'
                }
            
            ]
            
        }
    }
TrainingJobDefinition={
        'AlgorithmSpecification': {
            'TrainingImage': training_image,
            'TrainingInputMode': 'File',
            
        },
        'RoleArn': role,
        'StaticHyperParameters': [{
            "prediction_length": '50',
            "time_freq": 'min',
            "test_quantiles": '[0.5, 0.9]'
#            "_tuning_objective_metric": 'test:RMSE'  # can't figure out where this call needs to go.
        }],
        'InputDataConfig': [
            {
                'ChannelName': 'train',
                'DataSource': {
                    'S3DataSource': {
                        'S3DataType': 'S3Prefix',
                        'S3Uri': s3_input_train,
                        'S3DataDistributionType': 'FullyReplicated'
                    }
                },
                'ContentType': 'json',
                'CompressionType': 'None',
                'RecordWrapperType': 'None'
            ,
                
            'ChannelName': 'test',
                'DataSource': {
                    'S3DataSource': {
                        'S3DataType': 'S3Prefix',
                        'S3Uri': s3_input_test,
                        'S3DataDistributionType': 'FullyReplicated'
                    }
                },
                'ContentType': 'json',
                'CompressionType': 'None',
                'RecordWrapperType': 'None'

            }
        ],

        'OutputDataConfig': {
            "S3OutputPath": "s3://{}/{}/output".format(bucket,prefix)
        },
        'ResourceConfig': {
            'InstanceType': 'ml.m4.xlarge', # depending on size of data and number of TS, might need larger or smaller
            'InstanceCount': 1, #  You can change this limit, I had to request a limit
            'VolumeSizeInGB': 1,
        },
    'StoppingCondition': {
            'MaxRuntimeInSeconds': 86400
        }
        
}

## Run tuning job

In [ ]:
smclient.create_hyper_parameter_tuning_job(HyperParameterTuningJobName = 'TUNING-JOB-NAME',
                                            HyperParameterTuningJobConfig = HyperParameterTuningJobConfig,
                                            TrainingJobDefinition = TrainingJobDefinition1)

## Above command will run tuning job where you can access the best job parameters on the GUI or call them here to use for model training.